In [ ]:
import json
import os
import pandas as pd
from matplotlib import pyplot as plt
from filter_optimization.filter_optimization_task import extract_biomass_data

plt.rcParams['font.size'] = 18

In [ ]:
df = extract_biomass_data(56, '2020-06-01', '2020-08-31', 0.99)
df.hour = (df.hour + 2) % 24
plt.figure(figsize=(10, 5))
mask = df.estimated_k_factor > 0.0
df[mask].groupby('hour')['estimated_weight_g'].agg(lambda x: x.mean()).plot()
plt.ylabel('Average Weight (g)')
plt.grid()

In [ ]:
df = extract_biomass_data(60, '2020-06-01', '2020-08-31', 0.99)
df.hour = (df.hour + 2) % 24
plt.figure(figsize=(10, 5))
mask = df.estimated_k_factor > 0.0
df[mask].groupby('hour')['estimated_weight_g'].agg(lambda x: x.mean()).plot()
plt.grid()

In [ ]:
df = extract_biomass_data(100, '2020-06-01', '2020-08-31', 0.99)
df.hour = (df.hour + 2) % 24
plt.figure(figsize=(10, 5))
mask = df.estimated_k_factor > 0.0
df[mask].groupby('hour')['estimated_weight_g'].agg(lambda x: x.mean()).plot()
plt.grid()

In [ ]:
df = extract_biomass_data(88, '2020-02-01', '2020-03-06', 0.99)
df.hour = (df.hour + 2) % 24
plt.figure(figsize=(10, 5))
mask = df.estimated_k_factor > 0.0
df[mask].groupby('hour')['estimated_weight_g'].agg(lambda x: x.mean()).plot()
plt.grid()

In [ ]:
df = extract_biomass_data(95, '2020-04-01', '2020-07-30', 0.99)
df.hour = (df.hour + 2) % 24
plt.figure(figsize=(10, 5))
mask = df.estimated_k_factor > 0.0
df[mask].groupby('hour')['estimated_weight_g'].agg(lambda x: x.mean()).plot()
plt.grid()

In [ ]:
df = extract_biomass_data(131, '2020-08-12', '2020-08-26', 0.99)
df.hour = (df.hour + 2) % 24
plt.figure(figsize=(10, 5))
mask = df.estimated_k_factor > 0.0
df[mask].groupby('hour')['estimated_weight_g'].agg(lambda x: x.mean()).plot()
plt.grid()

In [ ]:
df = extract_biomass_data(132, '2020-08-12', '2020-08-26', 0.99)
df.hour = (df.hour + 2) % 24
plt.figure(figsize=(10, 5))
mask = df.estimated_k_factor > 0.0
df[mask].groupby('hour')['estimated_weight_g'].agg(lambda x: x.mean()).plot()
plt.grid()

In [ ]:
df = extract_biomass_data(131, '2020-06-01', '2020-08-31', 0.99)

plt.figure(figsize=(20, 10))
mask = df.estimated_k_factor > 1.08
df[mask].groupby('hour')['estimated_weight_g'].agg(lambda x: x.mean()).plot()
plt.grid()

In [ ]:
df = extract_biomass_data(132, '2020-06-01', '2020-08-31', 0.99)

plt.figure(figsize=(20, 10))
mask = df.estimated_k_factor > 1.08
df[mask].groupby('hour')['estimated_weight_g'].agg(lambda x: x.mean()).plot()
plt.grid()

In [ ]:
df[mask].groupby('hour')['estimated_weight_g'].agg(lambda x: x.shape[0]).plot()

In [ ]:
from research.weight_estimation.keypoint_utils.optics import pixel2world
import numpy as np
from research.weight_estimation.keypoint_utils import body_parts

def get_raw_3d_coordinates(keypoints, cm):
    wkps = pixel2world([item for item in keypoints['leftCrop'] if item['keypointType'] != 'BODY'],
                       [item for item in keypoints['rightCrop'] if item['keypointType'] != 'BODY'],
                       cm)

    all_wkps = np.array([list(wkps[bp]) for bp in body_parts.core_body_parts])
    return all_wkps

In [ ]:
depths = []
for idx, row in df.iterrows():
    try:
        wkps = get_raw_3d_coordinates(row.annotation, row.camera_metadata)
        depth = np.median(wkps[:, 1])
    except:
        depth = None
    depths.append(depth)
    

In [ ]:
df['depth'] = depths

In [ ]:
plt.hist(df.depth, bins=20)
plt.show()

In [ ]:
plt.hist(df.depth, bins=20)
plt.show()

In [ ]:
plt.hist(df.depth, bins=20)
plt.show()

In [ ]:
df = extract_biomass_data(124, '2020-08-10', '2020-08-29', 0.99)

plt.figure(figsize=(20, 10))
mask = df.estimated_k_factor > 1.25
df[mask].groupby('hour')['estimated_weight_g'].agg(lambda x: x.mean()).plot()
plt.grid()

In [ ]:
df = extract_biomass_data(124, '2020-08-10', '2020-08-29', 0.99)

plt.figure(figsize=(15, 8))
mask = df.estimated_k_factor > 1.3
df[mask].groupby('hour')['estimated_weight_g'].agg(lambda x: x.mean()).plot()
plt.grid()

In [ ]:
plt.figure(figsize=(15, 8))
df[mask].groupby('hour')['estimated_weight_g'].agg(lambda x: x.shape[0]).plot()
plt.grid()

In [ ]:
rds